In [59]:
import pandas as pd

In [60]:
building_permits_2023 = pd.read_csv("Data/Building_Permits_in_2023.csv")
building_permits_2022 = pd.read_csv("Data/Building_Permits_in_2022.csv")
building_permits = pd.concat(
    [
        building_permits_2022,
        building_permits_2023,
    ],
    axis=0,
)


In [61]:
print(building_permits.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 58093 entries, 0 to 10185
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   X                            58093 non-null  float64
 1   Y                            58093 non-null  float64
 2   OBJECTID                     58093 non-null  int64  
 3   DCRAINTERNALNUMBER           58093 non-null  float64
 4   ISSUE_DATE                   58093 non-null  object 
 5   PERMIT_ID                    58093 non-null  object 
 6   PERMIT_TYPE_NAME             58093 non-null  object 
 7   PERMIT_SUBTYPE_NAME          57246 non-null  object 
 8   PERMIT_CATEGORY_NAME         3334 non-null   object 
 9   APPLICATION_STATUS_NAME      58093 non-null  object 
 10  FULL_ADDRESS                 58093 non-null  object 
 11  DESC_OF_WORK                 22961 non-null  object 
 12  SSL                          56261 non-null  object 
 13  ZONING          

In [62]:
building_permits["PERMIT_CATEGORY_NAME"].unique()

array([nan, 'AFTER HOURS', 'EXTENSION', 'HISTORIC PROPERTY',
       'SOIL BORING', 'SCAFFOLDING', 'ANTENNA', 'TOWER CRANE',
       'FUEL BURNING', 'UNDERGROUND STORAGE TANK', 'TENT',
       'MINIATURE BOILER'], dtype=object)

In [63]:
building_permits = building_permits[building_permits["APPLICATION_STATUS_NAME"] == 'PERMIT ISSUED']

In [64]:
building_permits.shape

(48301, 44)

In [65]:
building_permits.isnull().sum()

X                                  0
Y                                  0
OBJECTID                           0
DCRAINTERNALNUMBER                 0
ISSUE_DATE                         0
PERMIT_ID                          0
PERMIT_TYPE_NAME                   0
PERMIT_SUBTYPE_NAME              845
PERMIT_CATEGORY_NAME           45017
APPLICATION_STATUS_NAME            0
FULL_ADDRESS                       0
DESC_OF_WORK                   28182
SSL                             1764
ZONING                          3662
PERMIT_APPLICANT               25473
FEE_TYPE                        4438
FEES_PAID                          0
OWNER_NAME                      2672
LASTMODIFIEDDATE                   0
CITY                           48301
STATE                          48301
LATITUDE                           0
LONGITUDE                          0
XCOORD                             0
YCOORD                             0
ZIPCODE                        48301
MARADDRESSREPOSITORYID             0
D

In [66]:
# dropping entirely null columns
building_permits.dropna(axis=1, how='all', inplace=True)

In [67]:
# Remove unnecessary columns
columns_to_drop = [
    "OBJECTID", # Not Useful in the analysis
    "GLOBALID", # Not Useful in the analysis
    "CREATED_USER", # Not Useful in the analysis
    "LAST_EDITED_USER", # Not Useful in the analysis
    'DCRAINTERNALNUMBER', # Not Useful in the analysis
    'APPLICATION_STATUS_NAME', # As we are keeping obeservations APPLICATION_STATUS_NAME == 'Permit Issued' 
    'BUSINESSIMPROVEMENTDISTRICT', # mostly nulls

]
building_permits.drop(columns=columns_to_drop, inplace=True)

In [68]:
building_permits.columns

Index(['X', 'Y', 'ISSUE_DATE', 'PERMIT_ID', 'PERMIT_TYPE_NAME',
       'PERMIT_SUBTYPE_NAME', 'PERMIT_CATEGORY_NAME', 'FULL_ADDRESS',
       'DESC_OF_WORK', 'SSL', 'ZONING', 'PERMIT_APPLICANT', 'FEE_TYPE',
       'FEES_PAID', 'OWNER_NAME', 'LASTMODIFIEDDATE', 'LATITUDE', 'LONGITUDE',
       'XCOORD', 'YCOORD', 'MARADDRESSREPOSITORYID', 'WARD', 'ANC', 'SMD',
       'DISTRICT', 'PSA', 'NEIGHBORHOODCLUSTER', 'CREATED_DATE',
       'LAST_EDITED_DATE'],
      dtype='object')

In [69]:
building_permits['ISSUE_DATE'] = pd.to_datetime(building_permits['ISSUE_DATE'])
building_permits['LASTMODIFIEDDATE'] = pd.to_datetime(building_permits['LASTMODIFIEDDATE'])
building_permits['CREATED_DATE'] = pd.to_datetime(building_permits['CREATED_DATE'])
building_permits['LAST_EDITED_DATE'] = pd.to_datetime(building_permits['LAST_EDITED_DATE']) 

In [70]:
building_permits.isnull().sum()

X                             0
Y                             0
ISSUE_DATE                    0
PERMIT_ID                     0
PERMIT_TYPE_NAME              0
PERMIT_SUBTYPE_NAME         845
PERMIT_CATEGORY_NAME      45017
FULL_ADDRESS                  0
DESC_OF_WORK              28182
SSL                        1764
ZONING                     3662
PERMIT_APPLICANT          25473
FEE_TYPE                   4438
FEES_PAID                     0
OWNER_NAME                 2672
LASTMODIFIEDDATE              0
LATITUDE                      0
LONGITUDE                     0
XCOORD                        0
YCOORD                        0
MARADDRESSREPOSITORYID        0
WARD                          0
ANC                           0
SMD                           0
DISTRICT                      0
PSA                           0
NEIGHBORHOODCLUSTER           0
CREATED_DATE                  0
LAST_EDITED_DATE              0
dtype: int64

In [71]:
building_permits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48301 entries, 3 to 10185
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   X                       48301 non-null  float64            
 1   Y                       48301 non-null  float64            
 2   ISSUE_DATE              48301 non-null  datetime64[ns, UTC]
 3   PERMIT_ID               48301 non-null  object             
 4   PERMIT_TYPE_NAME        48301 non-null  object             
 5   PERMIT_SUBTYPE_NAME     47456 non-null  object             
 6   PERMIT_CATEGORY_NAME    3284 non-null   object             
 7   FULL_ADDRESS            48301 non-null  object             
 8   DESC_OF_WORK            20119 non-null  object             
 9   SSL                     46537 non-null  object             
 10  ZONING                  44639 non-null  object             
 11  PERMIT_APPLICANT        22828 non-null  o

In [72]:
duplicate_rows = building_permits.duplicated()
print(f"Number of duplicate rows: {duplicate_rows.sum()}")

Number of duplicate rows: 0


In [73]:
def get_duplicate_column_pairs(df):
    duplicate_column_pairs = {}
    for i in range(len(df.columns)):
        for j in range(i + 1, len(df.columns)):
            if df.iloc[:, i].equals(df.iloc[:, j]):
                duplicate_column_pairs[df.columns[j]] = df.columns[i]
    return duplicate_column_pairs

In [74]:
duplicate_column_pairs = get_duplicate_column_pairs(building_permits)
print(f"Duplicate column pairs: {duplicate_column_pairs}")

Duplicate column pairs: {'LAST_EDITED_DATE': 'CREATED_DATE'}


In [75]:
building_permits.drop(columns=['LAST_EDITED_DATE'], inplace=True)

In [76]:
building_permits.shape

(48301, 28)

In [77]:
building_permits.to_csv("Data/building_permits.csv")